In [15]:
import os
import torch
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
from transformers import AutoFeatureExtractor, AutoModelForImageClassification
from sklearn.metrics import classification_report, confusion_matrix

# Load the model and feature extractor
extractor = AutoFeatureExtractor.from_pretrained("rafalosa/diabetic-retinopathy-224-procnorm-vit")
model = AutoModelForImageClassification.from_pretrained("rafalosa/diabetic-retinopathy-224-procnorm-vit")

c:\Users\bryon\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\models\vit\feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [16]:
# Evaluation data folder path and mapping of folder names to labels
evaluation_data_folder = "images/evaluation2/"
label_mapping = {
    "No_DR": 0,
    "Mild": 1,
    "Moderate": 2,
    "Severe": 3,
    "Proliferate_DR": 4
}

# Reverse the label_mapping to get the original names from numbers
reverse_label_mapping = {v: k for k, v in label_mapping.items()}


In [20]:
# Function to generate classification report and confusion matrix for a given folder
def generate_metrics_for_folder(data_folder, label_mapping):
    true_labels = []
    predicted_labels = []

    for image_filename in os.listdir(data_folder):
        image_path = os.path.join(data_folder, image_filename)
        image = Image.open(image_path)
        inputs = extractor(images=image, return_tensors="pt")
        outputs = model(**inputs)
        logits = outputs.logits
        probs = logits.softmax(dim=1)
        predicted_class_idx = torch.argmax(probs, dim=1).item()
        predicted_label = reverse_label_mapping.get(predicted_class_idx, "Unknown Label")

        folder_name = os.path.basename(data_folder)
        label = reverse_label_mapping.get(label_mapping.get(folder_name, -1), "Unknown Label")

        true_labels.append(label)
        predicted_labels.append(predicted_label)

    return true_labels, predicted_labels

# Initialize lists to accumulate true labels and predicted labels for the entire dataset
all_true_labels = []
all_predicted_labels = []

# Call the function for each folder to accumulate metrics for the entire dataset
for folder_name in os.listdir(evaluation_data_folder):
    folder_path = os.path.join(evaluation_data_folder, folder_name)
    true_labels, predicted_labels = generate_metrics_for_folder(folder_path, label_mapping)
    all_true_labels.extend(true_labels)
    all_predicted_labels.extend(predicted_labels)

# Generate classification report and confusion matrix for the entire dataset
report = classification_report(all_true_labels, all_predicted_labels, target_names=label_mapping.keys(), output_dict=True)
classification_df = pd.DataFrame(report).transpose()
classification_df = classification_df.drop("support", axis=1)

# Convert float values to percentages in the DataFrame
classification_df["precision"] = classification_df["precision"].apply(lambda x: f'{x:.2%}')
classification_df["recall"] = classification_df["recall"].apply(lambda x: f'{x:.2%}')
classification_df["f1-score"] = classification_df["f1-score"].apply(lambda x: f'{x:.2%}')

# Print the formatted classification report
print("Classification Report:")
print(classification_df)

print("Confusion Matrix:")
print(confusion_matrix(all_true_labels, all_predicted_labels))


Classification Report:
               precision  recall f1-score
No_DR             20.75%  73.33%   32.35%
Mild               6.82%  10.00%    8.11%
Moderate           0.00%   0.00%    0.00%
Severe             0.00%   0.00%    0.00%
Proliferate_DR     0.00%   0.00%    0.00%
accuracy          16.67%  16.67%   16.67%
macro avg          5.51%  16.67%    8.09%
weighted avg       5.51%  16.67%    8.09%
Confusion Matrix:
[[22  8  0  0  0]
 [27  3  0  0  0]
 [ 0 30  0  0  0]
 [28  2  0  0  0]
 [29  1  0  0  0]]


c:\Users\bryon\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\bryon\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\bryon\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,